In [1]:
# import torch and numpy and pretrained model
from torchvision import models
import torch
import numpy as np

# load pretrained model
pretrained_model = models.resnet50(weights = 'DEFAULT', progress = True)

In [2]:
# changes fully connected/classifier layer to new layer for us to train, 15 is the number of solutions or types of leaves
pretrained_model.fc = torch.nn.Linear(2048, 15)

In [3]:
# # Currently not in use, for if we wanna custom build a softmax but idk how to train it
# import torch.nn as nn
# # build custom softmax module
# class Softmax(nn.Module):
#     def __init__(self, n_inputs, n_outputs):
#         super().__init__()
#         self.linear = nn.Linear(n_inputs, n_outputs)
 
#     def forward(self, x):
#         pred = self.linear(x)
#         return pred

In [4]:
# # Currently not in use
# # adds softmax to model
# class MyModel(nn.Module):
#     def __init__(self, pretrained_model):
#         super(MyModel, self).__init__()
#         self.pretrained_model = pretrained_model
#         self.last_layer = Softmax(1000, n) # add how many nodes as input and output

#     def forward(self, x):
#         return self.last_layer(self.pretrained_model(x))

# model = MyModel(pretrained_model)

In [5]:
#freeze model except fc layer because we don't wanna retrain the pretrained model
for param in pretrained_model.parameters():
    param.requires_grad = False

for param in pretrained_model.fc.parameters():
    param.requires_grad = True

In [6]:
criterion = torch.nn.CrossEntropyLoss() #could write this out ourselves
# need to find an optimizer or make one for a custom softmax function
optimizer = torch.optim.SGD(pretrained_model.fc.parameters(), lr=0.001, momentum=0.9)

In [7]:
#import data
from PIL import Image
import os
from sklearn.preprocessing import OneHotEncoder
data_dir = "/Users/stevenli/SigAida/data/images/" #change to data directory

# gets the label based on the number
def getLabel(s):
    labels = {1 : 'Ulmus carpinifolia', 
                2 : 'Acer', 
                3 : 'Salix aurita', 
                4 : 'Quercus', 
                5 : 'Alnus incan', 
                6 : 'Betula pubescens', 
                7 : 'Salix alba \'Sericea\'', 
                8 : 'Populus tremula', 
                9 : 'Ulmus glabra', 
                10 : 'Sorbus aucuparia', 
                11 : 'Salix sinerea', 
                12 : 'Populus', 
                13 : 'Tilia', 
                14 : 'Sorbus intermedia', 
                15 : 'Fagus silvatica'}
    return labels.get(int(s))

im = [] # images
la = [] # labels not in use, because you can't feed strings to gpu, need to feed tensors
ohe = [] #s toring int values for one hot encodings
joined = []

for f in os.listdir(data_dir):
    im.append(data_dir + f)
    val = int(f[:-9].replace('l','')) # removes last 9 letters replaces l with blank then gets label
    la.append(getLabel(val)) 
    ohe.append(val)

#one hot encoding (changing 1-15 to tensors for gpu)
encoded = OneHotEncoder(categories = [[x for x in range(1, 16)]], sparse = False).fit_transform(np.array(ohe).reshape((len(ohe),1)))
print(encoded)

#joining image and label for organization and if we wanna shuffle
for image, label in zip(im, encoded):
    joined.append([image, label])

print(la)
print(im)
print(joined[1])

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
['Populus tremula', 'Ulmus glabra', 'Ulmus glabra', 'Populus tremula', 'Populus tremula', 'Ulmus glabra', 'Sorbus aucuparia', 'Salix sinerea', 'Salix sinerea', 'Sorbus aucuparia', 'Salix aurita', 'Acer', 'Acer', 'Salix aurita', 'Ulmus carpinifolia', 'Ulmus carpinifolia', 'Populus', 'Ulmus carpinifolia', 'Tilia', 'Tilia', 'Populus', 'Alnus incan', 'Quercus', 'Betula pubescens', 'Fagus silvatica', 'Sorbus intermedia', "Salix alba 'Sericea'", 'Sorbus intermedia', 'Fagus silvatica', 'Betula pubescens', "Salix alba 'Sericea'", "Salix alba 'Sericea'", 'Betula pubescens', "Salix alba 'Sericea'", 'Betula pubescens', 'Betula pubescens', "Salix alba 'Sericea'", 'Sorbus intermedia', "Salix alba 'Sericea'", 'Fagus silvatica', 'Fagus silvatica', 'Betula pubescens', 'Sorbus intermedia', 'Quercus', 'Alnus incan', 'Tilia', 'Populus', 'Ulmus carpini

/Users/stevenli/opt/anaconda3/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [8]:
# some preprocessing, randomize data, split data, load into dataloaders
import random

# splits (train, val, test) test currently not in use
split_ratio=(0.8, 0.1, 0.1)

def create_splits(data, split_ratio):
    random.shuffle(data)
    train = [data[i] for i in range(0, round(.8 * len(data)))]
    val = [data[i] for i in range(round(.8 * len(data)), round(.9 * len(data)))]
    test = [data[i] for i in range(round(.9 * len(data)), len(data))]
    return train, val, test

train, val, test = create_splits(joined, split_ratio)

print(len(joined))
print(len(train))
print(len(val))
print(len(test))

1125
900
112
113


In [9]:
#making dataset
from torch.utils.data import Dataset

# makes a custom dataset based on pytorch dataset class
class PlantDataset(Dataset):
    def __init__(self, data_dir, arr, transform = None):
        # initialize some valuess
        self.data_dir = data_dir
        self.data = [x[0] for x in arr]
        self.arr = arr

        #transform to normalize/resize all images
        self.transform = transform
    
    def __len__(self):
        return len(self.arr)
    
    def __getitem__(self, idx):
        img = self.transform(Image.open(self.arr[idx][0]))
        s = self.arr[idx][1]
        #returns a tuple of the transformed image and the label (one-hot encoding)
        return (img, s)

In [10]:
import torchvision.transforms as transforms

#defining our transforms
transform = transforms.Compose([
    transforms.Resize(224), # change to what data should be
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

#making dataset and dataloader (gives data to model, using batch_size 1 cuz google says that's good for sgd (stochastic gradient descent))
train_dataset = PlantDataset(data_dir, test, transform = transform)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=1, shuffle=True)

In [11]:
#chcek devices change to whatever you're using

#mps (m1/m2 mac) gpu

print(torch.backends.mps.is_available())
print(torch.backends.mps.is_built())

True
True


In [15]:
from tqdm import tqdm # progress

#store loss
loss_per_batch = []

#set device
device = torch.device('mps')

# Train the model
num_epochs = 30

for epoch in range(num_epochs):
    for data, label in tqdm(train_loader):
        # Move tensors to the configured device
        pretrained_model.to(device)
        images = data.to(device)
        labels = label.float().to(device) #.float() for some float64, float32 conversion thing, don't completely understand

        # Forward pass
        outputs = pretrained_model(images) #currently image tensor don't match fc layer if you use non pretrained model
        loss = criterion(outputs, labels) #calculate loss

        # Backward and optimize don't really understand this stuff
        optimizer.zero_grad()
        loss.backward()
        loss_per_batch.append(loss.item())
        optimizer.step()

        # Print some statistics
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

  0%|          | 0/113 [00:00<?, ?it/s]

100%|██████████| 113/113 [00:06<00:00, 16.99it/s]


Epoch [1/30], Loss: 1.0364


100%|██████████| 113/113 [00:04<00:00, 25.39it/s]


Epoch [2/30], Loss: 1.4476


...

Epoch [28/30], Loss: 0.2603


100%|██████████| 113/113 [00:04<00:00, 22.74it/s]


Epoch [29/30], Loss: 0.0155


100%|██████████| 113/113 [00:04<00:00, 23.44it/s]

Epoch [30/30], Loss: 0.1451


In [13]:
#loading validation dataset
val_dataset = PlantDataset(data_dir, val, transform = transform)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=1, shuffle=True)

In [16]:
total_correct = 0
total_instances = 0

pretrained_model.eval()
# iterating through batches without updating gradients
with torch.no_grad():
    for images, labels in tqdm(val_loader):
      images = images.to(device)
      # labels = labels.float().to(device) # don't need this cuz we not training no more

      # making classifications and deriving indices of maximum value via argmax (which gives the max value i the tensor)
      solution_tensor = pretrained_model(images)
      classifications = torch.argmax(solution_tensor, dim = 1).item()

      #undoing one-hot encoding to get label value as a number
      label = np.where(labels.numpy() == 1)[1]

      correct_predictions = int(classifications==label)

      #  incrementing counters
      total_correct+=correct_predictions
      total_instances+=len(images)

#print accuracy
print(round(total_correct/total_instances, 3))

100%|██████████| 112/112 [00:08<00:00, 13.92it/s]

0.902
